In [9]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np
# Store excel file locations to variables

Com_2013 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\11. S6 Services and status of the health facility\2013_s6_focus.csv"
Com_2014 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\11. S6 Services and status of the health facility\2014_s8_community.csv"

# Read excel files 
df_2013 = pd.read_csv(Com_2013)
df_2014 = pd.read_csv(Com_2014)



In [10]:
#This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.

# Rename columns in df to df_2013 column names
df_2013.rename(columns={
    'facility_id': 's8_health_id',
    'health_type': 's8_health_type',
    's6_q1': 's8_q1',
    's6_q2': 's8_q2',
    's6_q3': 's8_q3',
    's6_q4': 's8_q4',
    's6_q5': 's8_q5',
    's6_q6': 's8_q6',
    's6_q7': 's8_q9',
    's6_q8': 's8_q10',
    's6_q9': 's8_q11',
    's6_q10': 's8_q12'
}, inplace=True)



# df_2014 doesn't need renaming as it is the reference

In [11]:
# Define the column mappings based on the provided positions
# Make dictionaries for each year with updated names
# replace with 'None' where there are no columns
#Here are the updated mapping lists for the given datasets:


mapping_2013 = [
    'cid', 's8_health_id', 's8_health_type', 's8_q1', 's8_q2', 's8_q3',
    's8_q4', 's8_q5', 's8_q6', None, None, 's8_q9', 's8_q10',
    's8_q11', 's8_q12', None, None, None
]


mapping_2014 = [
    'cid', 's8_health_id', 's8_health_type', 's8_q1', 's8_q2', 's8_q3',
    's8_q4', 's8_q5', 's8_q6', 's8_q7', 's8_q8', 's8_q9', 's8_q10',
    's8_q11', 's8_q12', 's8_q13', 's8_q14', 's8_q15'
]





In [12]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [13]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [14]:
# Usage with dataframes and mappings
dfs = [ df_2013, df_2014]
mappings = [ mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0', 'cid', 's8_health_id', 's8_health_type', 's8_q1', 's8_q2', 's8_q3', 's8_q4', 's8_q5', 's8_q6', 's8_q9', 's8_q10', 's8_q11', 's8_q12']
Appending data for column cid
Appending data for column s8_health_id
Appending data for column s8_health_type
Appending data for column s8_q1
Appending data for column s8_q2
Appending data for column s8_q3
Appending data for column s8_q4
Appending data for column s8_q5
Appending data for column s8_q6
Appending data for column s8_q9
Appending data for column s8_q10
Appending data for column s8_q11
Appending data for column s8_q12
Processing DataFrame with columns: ['Unnamed: 0', 'cid', 's8_health_id', 's8_health_type', 's8_q1', 's8_q2', 's8_q3', 's8_q4', 's8_q5', 's8_q6', 's8_q7', 's8_q8', 's8_q9', 's8_q10', 's8_q11', 's8_q12', 's8_q13', 's8_q14', 's8_q15']
Appending data for column cid
Appending data for column s8_health_id
Appending data for column s8_health_type
Appending data for column s8_q1
Appending

In [15]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'cid': 'CID',
    's8_health_id': 'HF_ID',
    's8_health_type': 'HF_Type',
    's8_q1': 'HF_Status',
    's8_q2': 'HF_Imune',
    's8_q3': 'HF_Dia',
    's8_q4': 'HF_FPS',
    's8_q5': 'HF_DoBabies',
    's8_q6': 'HF_ANC',
    's8_q7': 'HF_EOC',
    's8_q8': 'HF_PNC',
    's8_q9': 'HF_MT',
    's8_q10': 'HF_MS',
    's8_q11': 'HF_MI',
    's8_q12': 'HF_TMInj',
    's8_q13': 'HF_SupA_Med',
    's8_q14': 'HF_SupA_MedEqp',
    's8_q15': 'HF_SupA_MedPers'
}


merged_df.rename(columns=rename_mapping, inplace=True)

# Drop redundant columns
df_2014.drop(df_2014.columns[df_2014.columns.str.contains('Unnamed', case=True)], axis=1, inplace=True)
df_2014.drop(df_2014.columns[df_2014.columns.str.contains(' ', case=False)], axis=1, inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('9. merged_S6_Services and status of the health facility.csv', index=False)

